In [127]:
import os
import pandas as pd
from datetime import datetime
import re

# 改變Date、File

In [17]:
data = pd.read_excel('加權指數.xlsx')

data['Date'] = pd.to_datetime(data['Date'], format='%Y/%m/%d')

for i in range(len(data.index)):
    name = data['Date'].iloc[i]
    data.loc[i, 'File'] = f"o{name.strftime('%Y%m%d')}.csv"

data['Date'] = data['Date'].dt.date
data.to_excel('加權指數.xlsx', index = False)

In [ ]:
data

# 算到期日

In [28]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd, Week

data = pd.read_excel('加權指數.xlsx')
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

def third_wednesday(date):
    last_day = (date + MonthEnd(0)).replace(day=1)
    first_wednesday = last_day + Week(weekday=2)
    if first_wednesday.day <= 7:
        return first_wednesday + Week(2)
    else:
        return first_wednesday + Week(1)

def update_contract_and_maturity(row):
    if row['Maturity'] < 0:
        next_month = row['Date'] + pd.DateOffset(months=1)
        maturity_date = third_wednesday(next_month)
        row['Contract'] = maturity_date.strftime('%Y-%m')
        row['Maturity'] = (maturity_date - row['Date']).days
    return row

data['MaturityDate'] = data['Date'].apply(third_wednesday)

data['Maturity'] = (data['MaturityDate'] - data['Date']).dt.days

data['Contract'] = data['MaturityDate'].dt.strftime('%Y-%m')

data = data.drop(columns=['MaturityDate'])

data = data.apply(update_contract_and_maturity, axis=1)

data['Date'] = data['Date'].dt.date

data.to_excel('加權指數.xlsx', index = False)

# 換檔名

In [129]:
import os
import re

folder_path = '/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/midterm/逐筆資料/Option_2023'

for filename in os.listdir(folder_path):
    match = re.search(r'OptionsDaily_(\d{4})_(\d{2})_(\d{2})', filename)
    if match:
        year, month, day = match.groups()
        new_filename = f'o{year}{month}{day}.csv'
        old_file_path = os.path.join(folder_path, filename)
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)
        print(f'Renamed: {filename} to {new_filename}')

Renamed: OptionsDaily_2023_04_10.csv to o20230410.csv
Renamed: OptionsDaily_2023_03_31.csv to o20230331.csv
Renamed: OptionsDaily_2023_06_01.csv to o20230601.csv
Renamed: OptionsDaily_2023_06_15.csv to o20230615.csv
Renamed: OptionsDaily_2023_06_29.csv to o20230629.csv
Renamed: OptionsDaily_2023_06_28.csv to o20230628.csv
Renamed: OptionsDaily_2023_06_14.csv to o20230614.csv
Renamed: OptionsDaily_2023_01_09.csv to o20230109.csv
Renamed: OptionsDaily_2023_03_24.csv to o20230324.csv
Renamed: OptionsDaily_2023_03_30.csv to o20230330.csv
Renamed: OptionsDaily_2023_04_11.csv to o20230411.csv
Renamed: OptionsDaily_2023_04_13.csv to o20230413.csv
Renamed: OptionsDaily_2023_04_07.csv to o20230407.csv
Renamed: OptionsDaily_2023_06_16.csv to o20230616.csv
Renamed: OptionsDaily_2023_06_02.csv to o20230602.csv
Renamed: OptionsDaily_2023_03_27.csv to o20230327.csv
Renamed: OptionsDaily_2023_04_06.csv to o20230406.csv
Renamed: OptionsDaily_2023_04_12.csv to o20230412.csv
Renamed: OptionsDaily_2023_0

# 解壓縮

In [124]:
folder_path = '/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/midterm/逐筆資料/Option_2023'

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        data = pd.read_csv(file_path, encoding = 'big5', low_memory = False)
        data.to_csv(file_path, index = False)
        print(filename)

    except UnicodeDecodeError as e:
        print(f"編碼錯誤（Big5 無法解碼）：{filename}，錯誤訊息：{e}")

    except Exception as e:
        print(f"處理檔案 {filename} 時發生錯誤：{e}")

o20231106.csv
OptionsDaily_2023_04_10.csv
OptionsDaily_2023_03_31.csv
OptionsDaily_2023_06_01.csv
OptionsDaily_2023_06_15.csv
o20230807.csv
OptionsDaily_2023_06_29.csv
OptionsDaily_2023_06_28.csv
OptionsDaily_2023_06_14.csv
OptionsDaily_2023_01_09.csv
OptionsDaily_2023_03_24.csv
OptionsDaily_2023_03_30.csv
OptionsDaily_2023_04_11.csv
o20231113.csv
o20231107.csv
OptionsDaily_2023_04_13.csv
OptionsDaily_2023_04_07.csv
OptionsDaily_2023_06_16.csv
o20230804.csv
o20230810.csv
OptionsDaily_2023_06_02.csv
o20230811.csv
OptionsDaily_2023_03_27.csv
OptionsDaily_2023_04_06.csv
OptionsDaily_2023_04_12.csv
o20231110.csv
o20231128.csv
o20231114.csv
OptionsDaily_2023_03_23.csv
o20230829.csv
OptionsDaily_2023_06_13.csv
o20230801.csv
o20230815.csv
OptionsDaily_2023_06_07.csv
編碼錯誤（Big5 無法解碼）：.DS_Store，錯誤訊息：'big5' codec can't decode byte 0xff in position 350: illegal multibyte sequence
OptionsDaily_2023_06_06.csv
o20230814.csv
OptionsDaily_2023_06_12.csv
o20230828.csv
OptionsDaily_2023_04_17.csv
Options

# 整理逐日資料

In [125]:
folder_path = '/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/midterm/逐筆資料/Option_2023'

for filename in os.listdir(folder_path):
    try:
        file_path = os.path.join(folder_path, filename)
        print(filename)
        try:
            data = pd.read_csv(file_path, dtype = str, low_memory = False, encoding = 'utf-8', delimiter = ',')
        except:
            data = pd.read_csv(file_path, dtype = str, low_memory = False, encoding = 'latin1', delimiter = ',')

        data.columns = data.columns.str.strip()
        for col in data.columns:
            data[col] = data[col].str.strip()
        unnamed_cols = [col for col in data.columns if 'Unnamed' in col]
        columns_to_drop = unnamed_cols + ['開盤集合競價']
        data = data.drop(columns_to_drop, axis = 1, errors = 'ignore')
        mapping = {
                '成交日期': 'Date',
                '商品代號': 'ID',
                '履約價格': 'E',
                '到期月份(週別)': 'Maturity_contract',
                '買賣權別': 'Call_Put',
                '成交時間': 'Time',
                '成交價格': 'Pr',
                '成交數量(B or S)': 'Num',
            }
        data = data.rename(mapping, axis = 1)
        data['Maturity_contract'] = data['Maturity_contract'].str.replace('.0', '', regex = False)
        data = data.reset_index(drop = True)
        data = data.dropna()
        data.to_csv(file_path, index = False)
    except:
        pass

o20231106.csv
OptionsDaily_2023_04_10.csv
OptionsDaily_2023_03_31.csv
OptionsDaily_2023_06_01.csv
OptionsDaily_2023_06_15.csv
o20230807.csv
OptionsDaily_2023_06_29.csv
OptionsDaily_2023_06_28.csv
OptionsDaily_2023_06_14.csv
OptionsDaily_2023_01_09.csv
OptionsDaily_2023_03_24.csv
OptionsDaily_2023_03_30.csv
OptionsDaily_2023_04_11.csv
o20231113.csv
o20231107.csv
OptionsDaily_2023_04_13.csv
OptionsDaily_2023_04_07.csv
OptionsDaily_2023_06_16.csv
o20230804.csv
o20230810.csv
OptionsDaily_2023_06_02.csv
o20230811.csv
OptionsDaily_2023_03_27.csv
OptionsDaily_2023_04_06.csv
OptionsDaily_2023_04_12.csv
o20231110.csv
o20231128.csv
o20231114.csv
OptionsDaily_2023_03_23.csv
o20230829.csv
OptionsDaily_2023_06_13.csv
o20230801.csv
o20230815.csv
OptionsDaily_2023_06_07.csv
.DS_Store
OptionsDaily_2023_06_06.csv
o20230814.csv
OptionsDaily_2023_06_12.csv
o20230828.csv
OptionsDaily_2023_04_17.csv
OptionsDaily_2023_03_22.csv
o20231101.csv
o20231115.csv
o20231129.csv
o20231103.csv
o20231117.csv
OptionsDai

# 整合資料

In [179]:
d2020C = pd.read_csv('/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/midterm/VolatilityResault/Finalresult_2020_C.txt', sep = '\t', header = None, names = ['date', 'IV_C0', 'Std_C0'])
d2020P = pd.read_csv('/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/midterm/VolatilityResault/Finalresult_2020_P.txt', sep = '\t', header = None, names = ['date', 'IV_P0', 'Std_P0'])
d2021C = pd.read_csv('/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/midterm/VolatilityResault/Finalresult_2021_C.txt', sep = '\t', header = None, names = ['date', 'IV_C0', 'Std_C0'])
d2021P = pd.read_csv('/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/midterm/VolatilityResault/Finalresult_2021_P.txt', sep = '\t', header = None, names = ['date', 'IV_P0', 'Std_P0'])
d2022C = pd.read_csv('/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/midterm/VolatilityResault/Finalresult_2022_C.txt', sep = '\t', header = None, names = ['date', 'IV_C0', 'Std_C0'])
d2022P = pd.read_csv('/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/midterm/VolatilityResault/Finalresult_2022_P.txt', sep = '\t', header = None, names = ['date', 'IV_P0', 'Std_P0'])
d2023C = pd.read_csv('/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/midterm/VolatilityResault/Finalresult_2023_C.txt', sep = '\t', header = None, names = ['date', 'IV_C0', 'Std_C0'])
d2023P = pd.read_csv('/Users/xinc./Documents/GitHub/desktop-tutorial/ndhu/midterm/VolatilityResault/Finalresult_2023_P.txt', sep = '\t', header = None, names = ['date', 'IV_P0', 'Std_P0'])

for i in [d2020C, d2020P, d2021C, d2021P, d2022C, d2022P, d2023C, d2023P]:
    i['date'] = pd.to_datetime(i['date'])

data1 = pd.concat([d2020C, d2021C, d2022C, d2023C], axis = 0, ignore_index = True)
data2 = pd.concat([d2020P, d2021P, d2022P, d2023P], axis = 0, ignore_index = True)
df = pd.concat([data1, data2], axis = 1)

df.to_excel('Volatility.xlsx', index = True)

ValueError: cannot assemble with duplicate keys